# Setup for PyTorch Mask-RCNN

## Installs

### Installing PyTorch

The Python used:
* DSVM:  `/anaconda/envs/py35/bin/python`
* macOS:  `/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/`


In [10]:
# What Python

import sys
sys.executable

'/usr/bin/python3'

In [11]:
# # If not built for the system already, uncomment and install
! LC_ALL=C pip3 install torch==0.3.1 torchvision

    100% |################################| 496.4MB 3.4kB/s eta 0:00:01    35% |###########                     | 174.4MB 45.9MB/s eta 0:00:08    47% |###############                 | 236.2MB 31.7MB/s eta 0:00:09    71% |######################          | 353.5MB 31.3MB/s eta 0:00:05.8MB 35.5MB/s eta 0:00:04    84% |###########################     | 419.8MB 58.5MB/s eta 0:00:02    98% |############################### | 488.5MB 25.4MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ca/0d/f00b2885711e08bd71242ebe7b96561e6f6d01fdb4b9dcf4d37e2e13c5e1/torchvision-0.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/86/04/bd774106ae0ae1ada68c67efe89f1a16b2aa373cc2db15d974002a9f136d/numpy-1.15.4-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/cc/b6e4

In [6]:
! LC_ALL=C pip3 install setuptools

  Using cached https://files.pythonhosted.org/packages/82/a1/ba6fb41367b375f5cb653d1317d8ca263c636cff6566e2da1b0da716069d/setuptools-40.5.0-py2.py3-none-any.whl
You are using pip version 8.1.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
# What PyTorch

import torch
torch.__version__

'0.4.0'

## Building the extensions

Could also do:

```
! export PATH=/usr/local/cuda/bin:$PATH &&\
    export USE_OPENCV=1 &&\
    export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH &&\
    export LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH &&\
    export CUDA_BIN_PATH=/usr/local/cuda/bin &&\
    export CUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda/ &&\
    export CUDNN_LIB_DIR=/usr/local/cuda/lib64 &&\
    export CUDA_HOST_COMPILER=cc &&\
    export USE_CUDA=1 &&\
    export USE_NNPACK=1 &&\
    cd nms/src/cuda/ && nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC &&\
    cd ../../ &&\
    TORCH_CUDA_ARCH_LIST="3.5 5.2 6.0 6.1+PTX" TORCH_NVCC_FLAGS="-Xfatbin -compress-all" &&\
    CC=cc CXX=cc++ python3 build.py
```

In [9]:
%%bash

CUDA_ARCH="-gencode arch=compute_30,code=sm_30 \
           -gencode arch=compute_35,code=sm_35 \
           -gencode arch=compute_50,code=sm_50 \
           -gencode arch=compute_52,code=sm_52 \
           -gencode arch=compute_60,code=sm_60 \
           -gencode arch=compute_61,code=sm_61 \
	   -gencode arch=compute_70,code=sm_70 "


# Build RoIAlign
cd lib/roi_align/src/cuda
echo 'Compiling crop_and_resize kernels by nvcc...'
nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC $CUDA_ARCH
cd ../../
python3 build.py
cd ../..

# Build NMS
cd lib/nms/src/cuda
echo "Compiling nms kernels by nvcc..."
nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC $CUDA_ARCH
cd ../../
python3 build.py
cd ../..

Compiling crop_and_resize kernels by nvcc...
Including CUDA code.
/home/tpol/pytorch-mask-rcnn-samples/lib/roi_align
generating /tmp/tmp6t_yzm0h/_crop_and_resize.c
setting the current directory to '/tmp/tmp6t_yzm0h'
running build_ext
building '_crop_and_resize' extension
creating home
creating home/tpol
creating home/tpol/pytorch-mask-rcnn-samples
creating home/tpol/pytorch-mask-rcnn-samples/lib
creating home/tpol/pytorch-mask-rcnn-samples/lib/roi_align
creating home/tpol/pytorch-mask-rcnn-samples/lib/roi_align/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/home/tpol/.local/lib/python3.5/site-packages/torch/utils/ffi/../../lib/include -I/home/tpol/.local/lib/python3.5/site-packages/torch/utils/ffi/../../lib/include/TH -I/home/tpol/.local/lib/python3.5/site-packages/torch/utils/ffi/../../lib/include/THC -I/usr/local/cuda/include -I/usr/in

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/home/tpol/pytorch-mask-rcnn-samples/lib/roi_align/src/crop_and_resize.c: In function 'CropAndResizePerBox':
/home/tpol/pytorch-mask-rcnn-samples/lib/roi_align/src/crop_and_resize.c:30:0: warning: ignoring #pragma omp parallel [-Wunknown-pragmas]
     #pragma omp parallel for
 ^
In file included from /home/tpol/.local/lib/python3.5/site-packages/torch/utils/ffi/../../lib/include/TH/TH.h:4:0,
                 from /home/tpol/pytorch-mask-rcnn-samples/lib/nms/src/nms.c:1:
/home/tpol/pytorch-mask-rcnn-samples/lib/nms/src/nms.c: In function 'cpu_nms':
/home/tpol/pytorch-mask-rcnn-samples/lib/nms/src/nms.c:7:42: warning: passing argument 1 of 'THLongTensor_isContiguous' from incompatible pointer type [-Wincompatible-pointer-types]
     THArgCheck(THLongTensor_isContiguous(boxes), 2, "boxes must be contiguous");
                                          ^
/home/tpol/.local/lib/python3.5/site-packages/torch/utils/ffi/../../l

## `pycocotools` and `coco` libraries

In [15]:
! LC_ALL=C pip3 install -r requirements.txt

    100% |################################| 2.8MB 612kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ca/0d/f00b2885711e08bd71242ebe7b96561e6f6d01fdb4b9dcf4d37e2e13c5e1/torchvision-0.2.1-py2.py3-none-any.whl
    100% |################################| 430kB 3.7MB/s eta 0:00:01
    100% |################################| 25.3MB 66kB/s  eta 0:00:01    83% |##########################      | 21.0MB 39.5MB/s eta 0:00:01
    100% |################################| 3.3MB 504kB/s eta 0:00:01
    100% |################################| 71kB 14.1MB/s ta 0:00:01
    100% |################################| 276kB 5.2MB/s eta 0:00:01
    100% |################################| 261.0MB 6.5kB/s eta 0:00:01  9% |###                             | 24.5MB 27.3MB/s eta 0:00:09    49% |###############                 | 128.7MB 32.3MB/s eta 0:00:05    66% |#####################           | 174.1MB 32.8MB/s eta 0:00:03    74% |#######################         | 195.4MB 25.5MB/s eta 0:00:

    100% |################################| 696kB 2.4MB/s eta 0:00:01
    100% |################################| 1.9MB 910kB/s eta 0:00:01
    100% |################################| 112kB 11.8MB/s ta 0:00:01
    100% |################################| 102kB 12.9MB/s a 0:00:01
    100% |################################| 737kB 2.2MB/s eta 0:00:01
    100% |################################| 81kB 12.1MB/s ta 0:00:01
    100% |################################| 1.1MB 1.6MB/s eta 0:00:01
    100% |################################| 81kB 12.9MB/s ta 0:00:01
    100% |################################| 71kB 14.4MB/s ta 0:00:01
    100% |################################| 317kB 5.2MB/s eta 0:00:01
    100% |################################| 358kB 4.9MB/s eta 0:00:01
    100% |################################| 81kB 12.9MB/s ta 0:00:01
    100% |################################| 112kB 12.9MB/s ta 0:00:01
    100% |################################| 92kB 14.0MB/s ta 0:00:01
    100% |################

    100% |################################| 61kB 11.3MB/s ta 0:00:01
    100% |################################| 51kB 5.3MB/s eta 0:00:01
    100% |################################| 450kB 3.5MB/s eta 0:00:01
    100% |################################| 81kB 12.3MB/s ta 0:00:01
    100% |################################| 102kB 12.8MB/s a 0:00:01
    100% |################################| 71kB 13.8MB/s ta 0:00:01
    100% |################################| 706kB 2.6MB/s eta 0:00:01
    100% |################################| 112kB 12.9MB/s ta 0:00:01
    100% |################################| 61kB 13.8MB/s ta 0:00:01
    100% |################################| 307kB 5.4MB/s eta 0:00:01
    100% |################################| 296kB 5.8MB/s eta 0:00:01
    100% |################################| 92kB 13.8MB/s ta 0:00:01
    100% |################################| 61kB 12.0MB/s ta 0:00:01
    100% |################################| 51kB 13.2MB/s ta 0:00:01
    100% |###################

    100% |################################| 112kB 12.9MB/s ta 0:00:01
    100% |################################| 112kB 13.1MB/s ta 0:00:01
    100% |################################| 61kB 13.1MB/s ta 0:00:01
    100% |################################| 40kB 12.3MB/s ta 0:00:01
    100% |################################| 194kB 8.1MB/s eta 0:00:01
    100% |################################| 368kB 4.5MB/s eta 0:00:01
    100% |################################| 51kB 13.7MB/s ta 0:00:01
    100% |################################| 419kB 3.8MB/s eta 0:00:01
    100% |################################| 92kB 13.7MB/s ta 0:00:01
    100% |################################| 563kB 3.1MB/s eta 0:00:01
    100% |################################| 51kB 13.5MB/s ta 0:00:01
    100% |################################| 71kB 12.7MB/s ta 0:00:01
    100% |################################| 81kB 11.6MB/s ta 0:00:01
    100% |################################| 92kB 12.6MB/s ta 0:00:01
    100% |##################

    100% |################################| 122kB 11.6MB/s ta 0:00:01
    100% |################################| 102kB 12.1MB/s a 0:00:01
    100% |################################| 51kB 13.4MB/s ta 0:00:01
    100% |################################| 122kB 10.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for docopt ... done
  Stored in directory: /home/tpol/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
  Running setup.py bdist_wheel for clint ... done
  Stored in directory: /home/tpol/.cache/pip/wheels/4f/e9/45/223565e5b1a4b09e12c6de6f8ba7c2c0e9127dec17cf830f83
  Running setup.py bdist_wheel for crontab ... done
  Stored in directory: /home/tpol/.cache/pip/wheels/71/27/d1/6ec781a7157e2d7d82a57a888e117c581b7288112e734fcde3
  Running setup.py bdist_wheel for tablib ... done
  Stored in directory: /home/tpol/.cache/pip/wheels/ea/e4/b9/9eb46eaf91476d5dd88a5cf6f76f0884114eafe0b2ef24e2df
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directo

You are using pip version 8.1.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
%%bash

rm -fr coco
rm -fr cocoapi
rm -fr pycocotools

export CC=cc &&\
export CXX=cc++ &&\

git clone https://github.com/waleedka/coco.git &&\
# git clone https://github.com/cocodataset/cocoapi.git &&\
cd coco &&\
cd PythonAPI &&\
python3 setup.py build_ext --inplace &&\
cd ../../ &&\
ln -s coco/PythonAPI/pycocotools/ pycocotools

Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx
running build_ext
building 'pycocotools._mask' extension
creating build
creating build/temp.linux-x86_64-3.5
creating build/temp.linux-x86_64-3.5/pycocotools
creating build/common
cc -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/home/tpol/.local/lib/python3.5/site-packages/numpy/core/include -I../common -I/usr/include/python3.5m -c pycocotools/_mask.c -o build/temp.linux-x86_64-3.5/pycocotools/_mask.o -Wno-cpp -Wno-unused-function -std=c99
cc -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/home/tpol/.local/lib/python3.5/site-packages/numpy/core/include -I../common -I/usr/include/python3.5m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.5/../common/maskApi.o -Wno-cpp -Wno-unused-functi

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Cloning into 'coco'...


Note, here, for macOS, using Xcode 9.4.1 command line tools

In [ ]:
# # MacOS, see https://pytorch.org/ for other distributions
# ! pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

More relevant resolved GitHub issues:
* https://github.com/uber/horovod/issues/274#issuecomment-390813445
* https://github.com/pytorch/pytorch/issues/10234